# 00: Exploratory Data Analysis

This notebook provides an initial exploration of the auto insurance dataset to:
- Understand data quality and completeness
- Identify demographic distributions and potential bias patterns
- Explore relationships between features and the target variable
- Inform preprocessing decisions

In [ ]:
# Install dependencies (for Google Colab)
!pip install fairlearn seaborn -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed for reproducibility
np.random.seed(42)

# Create results directory if it doesn't exist (for saving visualizations)
Path('../results').mkdir(exist_ok=True)

## 1. Setup & Load Data

### 1.1 Google Drive Mount (Optional - for Colab only)
If running in Google Colab and data is stored in Google Drive, uncomment the following cell.


In [ ]:
# Uncomment the following lines if you need to mount Google Drive in Colab
# from google.colab import drive
# drive.mount('/content/drive')

### 1.2 Load Dataset

In [ ]:
# Load the dataset
# Option 1: Load from local directory (works for both local and Colab after upload)
try:
    file_path = '../data/AutoInsurance.csv'
    df = pd.read_csv(file_path)
    print(f"✓ Successfully loaded data from {file_path}")
except FileNotFoundError:
    # Option 2: Load from Colab content directory (after manual upload)
    try:
        file_path = '/content/AutoInsurance.csv'
        df = pd.read_csv(file_path)
        print(f"✓ Successfully loaded data from {file_path}")
    except FileNotFoundError:
        print(f"Error: The file was not found.")
        print("For local: Ensure '../data/AutoInsurance.csv' exists")
        print("For Colab: Upload the file using the folder icon in the left sidebar")
except Exception as e:
    print(f"An error occurred while loading the CSV: {e}")

### 1.3 Data Inspection

In [ ]:
# Data Inspection - Initial overview
if 'df' in locals():
    print("=" * 60)
    print("DATASET OVERVIEW")
    print("=" * 60)
    print(f"\nShape of the DataFrame: {df.shape}")
    print(f"Total records: {len(df):,}")
    print(f"Total columns: {len(df.columns)}")
    
    print("\n" + "=" * 60)
    print("COLUMN NAMES:")
    print("=" * 60)
    print(df.columns.tolist())
    
    print("\n" + "=" * 60)
    print("DATA TYPES AND NON-NULL VALUES:")
    print("=" * 60)
    df.info()
    
    print("\n" + "=" * 60)
    print("FIRST FEW ROWS:")
    print("=" * 60)
    display(df.head(10))
else:
    print("DataFrame 'df' not created. Please check the previous steps for file loading errors.")

## 2. Data Quality Assessment

### 2.1 Missing Values

In [ ]:
# Check for missing values
if 'df' in locals():
    missing_values = df.isnull().sum()
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    missing_df = pd.DataFrame({
        'Missing Count': missing_values, 
        'Missing Percentage (%)': missing_percentage
    })
    # Filter to show only columns with missing values, and sort them
    missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing Count', ascending=False)
    
    if not missing_df.empty:
        print("\nMissing Values and their Percentages:")
        display(missing_df)
    else:
        print("✓ No missing values found in the DataFrame.")
else:
    print("DataFrame 'df' not created. Cannot check for missing values.")

### 2.2 Duplicate Records

In [ ]:
# Check for duplicate records
if 'df' in locals():
    num_duplicates = df.duplicated().sum()
    if num_duplicates > 0:
        print(f"⚠ Found {num_duplicates} duplicate record(s).")
        print("Displaying the first 5 duplicate rows:")
        display(df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()).head())
    else:
        print("✓ No duplicate records found.")
else:
    print("DataFrame 'df' not created. Cannot check for duplicates.")

### 2.3 Outliers

Examine numerical columns for potential outliers using descriptive statistics.

In [ ]:
# Check for outliers in key numerical columns
if 'df' in locals():
    numerical_cols_for_outliers = ['Income', 'Total Claim Amount', 'Customer Lifetime Value']
    print(f"Descriptive Statistics for Outlier Identification:")
    for col in numerical_cols_for_outliers:
        if col in df.columns:
            print(f"\n{'='*60}")
            print(f"Column: {col}")
            print(f"{'='*60}")
            display(df[col].describe())
            
            # Box plot for visual outlier detection
            plt.figure(figsize=(8, 5))
            plt.boxplot(df[col].dropna())
            plt.title(f'Box Plot: {col}')
            plt.ylabel(col)
            plt.show()
        else:
            print(f"Column '{col}' not found in DataFrame.")
else:
    print("DataFrame 'df' not created. Cannot check for outliers.")

### 2.4 Data Types Verification

In [ ]:
# Verify data types and identify numerical vs categorical columns
if 'df' in locals():
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    print("Numerical Columns:")
    print(numerical_cols)
    print(f"\nTotal: {len(numerical_cols)} numerical columns")
    
    print("\n" + "="*60)
    print("Categorical Columns:")
    print(categorical_cols)
    print(f"\nTotal: {len(categorical_cols)} categorical columns")
else:
    print("DataFrame 'df' not created. Cannot verify data types.")

### 2.5 Unique Values (Cardinality) of Categorical Features

In [ ]:
# Check cardinality of categorical features
if 'df' in locals():
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    print("Unique values (Cardinality) for Categorical Columns:")
    print("="*60)
    for col in categorical_cols:
        unique_count = df[col].nunique()
        print(f"\n{col}: {unique_count} unique values")
        # Display actual unique values if there are few
        if unique_count < 20:
            print(f"  Values: {df[col].unique().tolist()}")
        elif unique_count == len(df):
            print(f"  ⚠ Note: This appears to be a unique identifier (one value per row)")
else:
    print("DataFrame 'df' not created. Cannot check unique values.")

## 3. Target Variable Analysis

In [ ]:
# Examine target variable (Response)
target_col = 'Response'

if 'df' in locals() and target_col in df.columns:
    print("="*60)
    print("TARGET VARIABLE DISTRIBUTION")
    print("="*60)
    
    response_distribution = df[target_col].value_counts()
    print("\nCount distribution:")
    display(response_distribution)
    
    response_percentage = df[target_col].value_counts(normalize=True) * 100
    print("\nPercentage distribution:")
    display(response_percentage)
    
    # Calculate class imbalance ratio
    if len(response_distribution) == 2:
        imbalance_ratio = response_distribution.max() / response_distribution.min()
        print(f"\nClass Imbalance Ratio (Majority to Minority): {imbalance_ratio:.2f}")
        if imbalance_ratio > 5:
            print("⚠ Warning: Significant class imbalance detected. Consider using class weights or resampling.")
    
    # Enhanced visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    sns.countplot(x=target_col, data=df, palette='viridis', ax=axes[0])
    axes[0].set_title('Distribution of Response Variable (Count)')
    axes[0].set_ylabel('Number of Customers')
    axes[0].set_xlabel('Response')
    
    # Pie chart for percentages
    response_counts = df[target_col].value_counts()
    axes[1].pie(response_counts.values, labels=response_counts.index, autopct='%1.1f%%', 
                startangle=90, colors=['#440154', '#21918c'])
    axes[1].set_title('Response Distribution (Percentages)')
    
    plt.tight_layout()
    
    # Optional: Save visualization (uncomment to save)
    # fig.savefig('../results/target_variable_distribution.png', dpi=300, bbox_inches='tight')
    # print("\n✓ Visualization saved to '../results/target_variable_distribution.png'")
    
    plt.show()
else:
    print("DataFrame 'df' not created or 'Response' column not found. Cannot perform target variable analysis.")

## 4. Protected Attributes Identification

Identify and visualize demographic/socioeconomic attributes that may be sources of bias.

In [ ]:
# Define protected attributes for fairness analysis
protected_attributes_categorical = [
    'Gender',                   # M/F - direct protected attribute
    'EmploymentStatus',         # Employed/Unemployed/etc. - socioeconomic status
    'Education',                # Educational attainment - socioeconomic
    'Marital Status',           # Marital status
    'Location Code',            # Urban/Suburban/Rural - potential proxy for SES
    'State'                    
]
protected_attributes_numerical = [
    'Income',
]

# Visualize distributions of categorical protected attributes
if 'df' in locals():
    print("="*60)
    print("PROTECTED ATTRIBUTES: CATEGORICAL DISTRIBUTIONS")
    print("="*60)
    
    for col in protected_attributes_categorical:
        if col in df.columns:
            plt.figure(figsize=(8, 5))
            sns.countplot(x=col, data=df, palette='viridis', order=df[col].value_counts().index)
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
            plt.ylabel('Count')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        else:
            print(f"Column '{col}' not found in DataFrame.")
    
    print("\n" + "="*60)
    print("PROTECTED ATTRIBUTES: NUMERICAL DISTRIBUTIONS")
    print("="*60)
    
    for col in protected_attributes_numerical:
        if col in df.columns:
            plt.figure(figsize=(8, 5))
            sns.histplot(df[col], bins=30, kde=True, color='skyblue')
            plt.title(f'Distribution of {col}')
            plt.xlabel(col)
            plt.ylabel('Frequency')
            plt.tight_layout()
            plt.show()
        else:
            print(f"Column '{col}' not found in DataFrame.")
else:
    print("DataFrame 'df' not created. Cannot visualize protected attributes.")

## 5. Cross-Tabulation Analysis

Analyze relationships between protected attributes and the target variable to identify potential bias patterns.

In [ ]:
# Cross-tabulation analysis for protected attributes vs Response
if 'df' in locals():
    # Safety check: ensure protected attributes are defined
    if 'protected_attributes_categorical' not in locals():
        protected_attributes_categorical = ['Gender', 'EmploymentStatus', 'Education', 'Marital Status', 'Location Code', 'State']
    if 'protected_attributes_numerical' not in locals():
        protected_attributes_numerical = ['Income']
    
    print("="*60)
    print("CROSS-TABULATION ANALYSIS: Protected Attributes vs Response")
    print("="*60)
    
    # Analyze categorical protected attributes
    for col in protected_attributes_categorical:
        if col in df.columns:
            print(f"\n{'='*60}")
            print(f"Analysis for {col}")
            print(f"{'='*60}")
            
            # Create crosstab
            crosstab_df = pd.crosstab(df[col], df['Response'], normalize='index') * 100
            print(f"Response Rates by {col} (%):")
            display(crosstab_df)
            
            # Visualize with stacked bar chart
            crosstab_df.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
            plt.title(f'Response Rates by {col}')
            plt.xlabel(col)
            plt.ylabel('Percentage')
            plt.xticks(rotation=45, ha='right')
            plt.legend(title='Response')
            plt.tight_layout()
            plt.show()
        else:
            print(f"Column '{col}' not found in DataFrame. Skipping.")
    
    # Handle numerical protected attribute 'Income' by binning
    if 'Income' in df.columns:
        print(f"\n{'='*60}")
        print("Analysis for Income (Binned)")
        print(f"{'='*60}")
        
        # Create income bins
        df['Income_Binned_Labels'] = pd.cut(df['Income'], bins=5, include_lowest=True).astype(str)
        
        # Create crosstab for binned income
        crosstab_income_df = pd.crosstab(df['Income_Binned_Labels'], df['Response'], normalize='index') * 100
        print("Response Rates by Income Group (%):")
        display(crosstab_income_df)
        
        # Visualize
        crosstab_income_df.plot(kind='bar', stacked=True, figsize=(10, 6), colormap='viridis')
        plt.title('Response Rates by Income Group')
        plt.xlabel('Income Group')
        plt.ylabel('Percentage')
        plt.xticks(rotation=45, ha='right')
        plt.legend(title='Response')
        plt.tight_layout()
        plt.show()
        
        # Drop the temporary binned income column
        df.drop(columns=['Income_Binned_Labels'], inplace=True)
    else:
        print("Column 'Income' not found in DataFrame. Skipping.")
else:
    print("DataFrame 'df' not created. Cannot perform cross-tabulation analysis.")

## 6. Feature-Target Relationships

In [ ]:
# Feature-Target Relationship Analysis
if 'df' in locals():
    # Convert 'Response' to numerical (0 for No, 1 for Yes) for correlation analysis
    df_encoded = df.copy()
    df_encoded['Response_Numerical'] = df_encoded['Response'].map({'No': 0, 'Yes': 1})
    
    numerical_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()
    # Ensure 'Response_Numerical' is in the list
    if 'Response_Numerical' not in numerical_cols:
        numerical_cols.append('Response_Numerical')
    
    # Calculate correlations with 'Response_Numerical'
    relevant_numerical_cols = [col for col in numerical_cols if col in df_encoded.columns]
    correlations = df_encoded[relevant_numerical_cols].corr()['Response_Numerical'].sort_values(ascending=False)
    
    print("="*60)
    print("CORRELATION OF NUMERICAL FEATURES WITH RESPONSE")
    print("="*60)
    display(correlations)
    
    # Visualize correlations with a heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(df_encoded[relevant_numerical_cols].corr(), annot=True, cmap='coolwarm', 
                fmt=".2f", center=0, vmin=-1, vmax=1)
    plt.title('Correlation Matrix of Numerical Features and Response')
    plt.tight_layout()
    
    # Optional: Save visualization (uncomment to save)
    # plt.savefig('../results/correlation_heatmap.png', dpi=300, bbox_inches='tight')
    # print("\n✓ Visualization saved to '../results/correlation_heatmap.png'")
    
    plt.show()
    
    print("\n" + "="*60)
    print("FEATURES WITH POTENTIALLY STRONG PREDICTIVE POWER")
    print("(Absolute correlation > 0.1):")
    print("="*60)
    strong_predictors = correlations[abs(correlations) > 0.1]
    display(strong_predictors)
    
    print("\n" + "="*60)
    print("POTENTIAL PROXIES FOR PROTECTED ATTRIBUTES")
    print("(Correlation with Income):")
    print("="*60)
    if 'Income' in relevant_numerical_cols:
        income_correlations = df_encoded[relevant_numerical_cols].corr()['Income'].sort_values(ascending=False)
        display(income_correlations[abs(income_correlations) > 0.2])  # Show only moderate+ correlations
        print("\n⚠ Note: Features strongly correlated with Income may act as proxies for socioeconomic status.")
else:
    print("DataFrame 'df' not created. Cannot perform feature-target relationship analysis.")

## 7. Summary Statistics

Overall descriptive statistics for numerical and categorical features.

In [ ]:
# Summary statistics for numerical and categorical features
if 'df' in locals():
    print("="*60)
    print("SUMMARY STATISTICS FOR NUMERICAL FEATURES")
    print("="*60)
    display(df.describe())
    
    print("\n" + "="*60)
    print("SUMMARY STATISTICS FOR CATEGORICAL FEATURES")
    print("="*60)
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Create summary table for categorical features
    categorical_summary = []
    for col in categorical_cols:
        value_counts = df[col].value_counts()
        categorical_summary.append({
            'Column': col,
            'Unique Values': df[col].nunique(),
            'Most Frequent': value_counts.index[0] if len(value_counts) > 0 else None,
            'Most Frequent Count': value_counts.iloc[0] if len(value_counts) > 0 else 0,
            'Most Frequent %': (value_counts.iloc[0] / len(df) * 100) if len(value_counts) > 0 else 0
        })
    
    categorical_summary_df = pd.DataFrame(categorical_summary)
    display(categorical_summary_df)
else:
    print("DataFrame 'df' not created. Cannot generate summary statistics.")

## 8. Key Findings & Preprocessing Recommendations

This section summarizes the key findings from the exploratory data analysis and provides recommendations for preprocessing.

### Data Quality Report

*   **Missing Values**: No missing values were found in the dataset, which simplifies the initial data cleaning process.
*   **Duplicate Records**: No duplicate records were found, indicating a clean dataset in terms of unique entries.
*   **Outliers**:
    *   **Income**: Contains a significant number of zero values (approximately 25%), which heavily skews its distribution. There are also high-income outliers.
    *   **Total Claim Amount**: Shows a right-skewed distribution with potential outliers on the higher end.
    *   **Customer Lifetime Value**: Exhibits a highly right-skewed distribution with a long tail, suggesting many high-value outliers.
    These outliers will need careful consideration (e.g., capping, transformation, or robust modeling techniques) during preprocessing.
*   **Data Types**: Verified as mostly appropriate.
    *   8 numerical columns (`float64`, `int64`)
    *   16 object (categorical) columns.
    *   'Effective To Date' is currently an `object` type and needs to be converted to `datetime` for proper temporal feature engineering.
*   **Cardinality**:
    *   'Customer' column has 9134 unique values, indicating it is a unique identifier. This column should be dropped as it provides no predictive power for the model.
    *   Other categorical columns have a reasonable number of unique values suitable for encoding.

### Demographic Imbalances Observed

Analysis of protected attributes revealed several demographic imbalances:

*   **Gender**: The distribution between 'Female' (F) and 'Male' (M) is relatively balanced.
*   **EmploymentStatus**: Heavily skewed towards 'Employed' (approx. 62%) and 'Unemployed' (approx. 25%), with 'Medical Leave', 'Disabled', and 'Retired' groups being significantly smaller.
*   **Education**: Skewed towards 'Bachelor' and 'College' degrees, with fewer individuals in 'High School or Below', 'Master', and 'Doctor' categories.
*   **Marital Status**: 'Married' individuals constitute the largest group (approx. 58%), followed by 'Single' and 'Divorced'.
*   **Location Code**: Predominantly 'Suburban' (approx. 63%), with 'Rural' and 'Urban' areas having fewer representatives.
*   **State**: 'California' and 'Oregon' are the most represented states, while 'Washington' and 'Nevada' have smaller populations in the dataset.
*   **Income**: The distribution is highly skewed, with a large concentration of zero incomes and a long tail extending to higher values.

These imbalances highlight the need for careful evaluation of model performance across different groups to ensure fairness.

### Initial Bias Patterns Identified (Cross-Tabulation Analysis with 'Response')

The cross-tabulation analysis of protected attributes against the 'Response' (target) variable revealed initial patterns that could indicate bias:

*   **EmploymentStatus**: 'Employed' and 'Retired' groups show noticeably higher 'Yes' response rates (approx. 17.7% and 16.9% respectively) compared to 'Unemployed' individuals (approx. 5.6%). This is a significant disparity that warrants further investigation for potential bias.
*   **Education**: Individuals with 'Doctor', 'Bachelor', and 'College' degrees tend to have higher 'Yes' response rates (15.5% - 17.3%) than those with 'High School or Below' (approx. 9.5%).
*   **Income**: Binned income analysis indicates that middle-income groups (e.g., (19996, 39992] and (59988, 79984]) have higher 'Yes' response rates (approx. 17.9% and 16.0%) compared to the lowest income bin (e.g., zero incomes) (approx. 12.6%) and the highest income bin (approx. 11.1%). The group with zero income (part of the lowest bin) has a particularly low response rate.
*   **Gender**: 'Male' customers show a slightly higher 'Yes' response rate (approx. 14.9%) compared to 'Female' customers (approx. 13.7%). While small, this difference should be monitored.
*   **Location Code**: 'Suburban' customers have a slightly higher 'Yes' response rate (approx. 15.1%) compared to 'Urban' (approx. 12.2%) and 'Rural' (approx. 13.3%) customers.
*   **State** and **Marital Status**: Differences in response rates across these attributes were less pronounced but still present.

These patterns suggest that certain demographic groups are more likely to respond 'Yes', which could lead to biased outcomes if not addressed.

### Feature-Target Relationships

*   **Numerical Features**: The correlation analysis between numerical features and the 'Response' target variable (converted to numerical) showed very weak linear relationships. The highest absolute correlations were around 0.01-0.02 (e.g., 'Total Claim Amount', 'Income', 'Monthly Premium Auto'). This indicates that individual numerical features, in isolation, are not strong linear predictors of the 'Response'.
*   **Potential Proxy Relationships**: 'Income' showed a moderate negative correlation with 'Total Claim Amount' (-0.35). While not a direct proxy for protected groups in the traditional sense, this inverse relationship is noteworthy. Further analysis might be needed to understand if 'Total Claim Amount' can indirectly reflect socioeconomic status or contribute to disparate impact.

### Preprocessing Recommendations

Based on the EDA, the following preprocessing steps are recommended:

1.  **Target Variable ('Response') Imbalance**:
    *   The 'Response' variable is highly imbalanced with a majority-to-minority ratio of **5.98** (85.7% 'No' vs 14.3% 'Yes').
    *   **Recommendation**: Address this imbalance using techniques such as oversampling the minority class (e.g., SMOTE), undersampling the majority class, or using class weights in model training.
    *   **Evaluation Metrics**: Prioritize metrics robust to imbalance, such as F1-score, Precision, Recall, and AUC-ROC, over simple accuracy.

2.  **Categorical Feature Encoding**:
    *   All categorical features (excluding 'Customer' and 'Effective To Date') will require encoding.
    *   **Recommendation**: Use appropriate encoding strategies like One-Hot Encoding for nominal features and Label Encoding or Ordinal Encoding for ordinal features, based on further domain knowledge or experimentation.

3.  **'Effective To Date' Column**:
    *   Currently an `object` type.
    *   **Recommendation**: Convert this column to `datetime` objects. Extract useful temporal features such as 'year', 'month', 'day of week', 'day of month', or 'days until policy expiration' (if policy expiration info is available/derivable), and then drop the original column.

4.  **'Customer' Column**:
    *   Identified as a unique identifier with no predictive value.
    *   **Recommendation**: Drop this column from the dataset before model training.

5.  **Outlier Treatment**:
    *   **Recommendation**: For 'Customer Lifetime Value', 'Income', and 'Total Claim Amount', explore strategies like capping (e.g., at the 95th or 99th percentile) or applying transformations (e.g., log transformation) to mitigate the impact of extreme outliers. Robust scaling methods can also be considered. The zero-income values might need special handling (e.g., imputation or treated as a separate category) if they represent a distinct group.

6.  **Fairness Considerations**:
    *   The identified demographic imbalances and disparate response rates across protected attributes (especially 'EmploymentStatus', 'Education', and 'Income' groups) indicate potential fairness issues.
    *   **Recommendation**: During model development, monitor fairness metrics (e.g., demographic parity, equal opportunity) across these protected groups. Consider using fair-aware machine learning techniques if biases persist in model predictions.

**Next Steps:**
- Proceed to `01_preprocessing.ipynb` for data cleaning and preparation, incorporating these recommendations.